In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import explained_variance_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier as dtc
from sklearn.metrics import classification_report

In [2]:
# reading a csv file

df_main = pd.read_csv('/kaggle/input/ip-network-traffic-flows-labeled-with-87-apps/Dataset-Unicauca-Version2-87Atts.csv')

In [3]:
df_main.head()

,Flow.ID,Source.IP,Source.Port,Destination.IP,Destination.Port,Protocol,Timestamp,Flow.Duration,Total.Fwd.Packets,Total.Backward.Packets,...,Active.Std,Active.Max,Active.Min,Idle.Mean,Idle.Std,Idle.Max,Idle.Min,Label,L7Protocol,ProtocolName
0,172.19.1.46-10.200.7.7-52422-3128-6,172.19.1.46,52422,10.200.7.7,3128,6,26/04/201711:11:17,45523,22,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,131,HTTP_PROXY
1,172.19.1.46-10.200.7.7-52422-3128-6,10.200.7.7,3128,172.19.1.46,52422,6,26/04/201711:11:17,1,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,131,HTTP_PROXY
2,10.200.7.217-50.31.185.39-38848-80-6,50.31.185.39,80,10.200.7.217,38848,6,26/04/201711:11:17,1,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,7,HTTP
3,10.200.7.217-50.31.185.39-38848-80-6,50.31.185.39,80,10.200.7.217,38848,6,26/04/201711:11:17,217,1,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,7,HTTP
4,192.168.72.43-10.200.7.7-55961-3128-6,192.168.72.43,55961,10.200.7.7,3128,6,26/04/201711:11:17,78068,5,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,131,HTTP_PROXY


In [4]:
df_main.shape

(3577296, 87)

In [5]:
df_main.columns

Index(['Flow.ID', 'Source.IP', 'Source.Port', 'Destination.IP',
       'Destination.Port', 'Protocol', 'Timestamp', 'Flow.Duration',
       'Total.Fwd.Packets', 'Total.Backward.Packets',
       'Total.Length.of.Fwd.Packets', 'Total.Length.of.Bwd.Packets',
       'Fwd.Packet.Length.Max', 'Fwd.Packet.Length.Min',
       'Fwd.Packet.Length.Mean', 'Fwd.Packet.Length.Std',
       'Bwd.Packet.Length.Max', 'Bwd.Packet.Length.Min',
       'Bwd.Packet.Length.Mean', 'Bwd.Packet.Length.Std', 'Flow.Bytes.s',
       'Flow.Packets.s', 'Flow.IAT.Mean', 'Flow.IAT.Std', 'Flow.IAT.Max',
       'Flow.IAT.Min', 'Fwd.IAT.Total', 'Fwd.IAT.Mean', 'Fwd.IAT.Std',
       'Fwd.IAT.Max', 'Fwd.IAT.Min', 'Bwd.IAT.Total', 'Bwd.IAT.Mean',
       'Bwd.IAT.Std', 'Bwd.IAT.Max', 'Bwd.IAT.Min', 'Fwd.PSH.Flags',
       'Bwd.PSH.Flags', 'Fwd.URG.Flags', 'Bwd.URG.Flags', 'Fwd.Header.Length',
       'Bwd.Header.Length', 'Fwd.Packets.s', 'Bwd.Packets.s',
       'Min.Packet.Length', 'Max.Packet.Length', 'Packet.Length.Mean',
  

In [6]:
# df_main = df_main.drop(['Flow.ID', 'Protocol', 'Timestamp', 
#                         'Fwd.Packet.Length.Std', 
#                         'Bwd.Packet.Length.Std',
#                         'Flow.IAT.Std', 'Fwd.IAT.Mean', 
#                         'Fwd.IAT.Std',
#                         'Bwd.IAT.Std', 'Bwd.PSH.Flags', 'Fwd.URG.Flags', 
#                         'Bwd.URG.Flags', 'Min.Packet.Length', 'Max.Packet.Length', 'Packet.Length.Std', 
#                         'Packet.Length.Variance', 
#                         'CWE.Flag.Count', 'Fwd.Avg.Bytes.Bulk', 
#                         'Fwd.Avg.Packets.Bulk', 'Fwd.Avg.Bulk.Rate', 'Bwd.Avg.Bytes.Bulk', 
#                         'Bwd.Avg.Packets.Bulk', 'Bwd.Avg.Bulk.Rate', 'min_seg_size_forward', 
#                         'Active.Std', 'Active.Max', 'Active.Min', 'Idle.Std', 'Idle.Max', 'Idle.Min', 
#                         'Label'], axis=1)

single_unique_cols = [col for col in df_main.columns if df_main[col].nunique() == 1]
df_main.drop(single_unique_cols, axis = 1, inplace = True)

df_main.drop(['Timestamp', 'Flow.ID'], axis = 1, inplace = True)

In [7]:
df_main.columns

Index(['Source.IP', 'Source.Port', 'Destination.IP', 'Destination.Port',
       'Protocol', 'Flow.Duration', 'Total.Fwd.Packets',
       'Total.Backward.Packets', 'Total.Length.of.Fwd.Packets',
       'Total.Length.of.Bwd.Packets', 'Fwd.Packet.Length.Max',
       'Fwd.Packet.Length.Min', 'Fwd.Packet.Length.Mean',
       'Fwd.Packet.Length.Std', 'Bwd.Packet.Length.Max',
       'Bwd.Packet.Length.Min', 'Bwd.Packet.Length.Mean',
       'Bwd.Packet.Length.Std', 'Flow.Bytes.s', 'Flow.Packets.s',
       'Flow.IAT.Mean', 'Flow.IAT.Std', 'Flow.IAT.Max', 'Flow.IAT.Min',
       'Fwd.IAT.Total', 'Fwd.IAT.Mean', 'Fwd.IAT.Std', 'Fwd.IAT.Max',
       'Fwd.IAT.Min', 'Bwd.IAT.Total', 'Bwd.IAT.Mean', 'Bwd.IAT.Std',
       'Bwd.IAT.Max', 'Bwd.IAT.Min', 'Fwd.PSH.Flags', 'Fwd.Header.Length',
       'Bwd.Header.Length', 'Fwd.Packets.s', 'Bwd.Packets.s',
       'Min.Packet.Length', 'Max.Packet.Length', 'Packet.Length.Mean',
       'Packet.Length.Std', 'Packet.Length.Variance', 'FIN.Flag.Count',
       'SYN.

In [8]:
print(df_main['L7Protocol'].nunique())
print(df_main['ProtocolName'].nunique())

78
78


In [9]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(df_main.groupby(['L7Protocol', 'ProtocolName']).size())

L7Protocol  ProtocolName     
1           FTP_CONTROL              25
5           DNS                    1695
7           HTTP                 683734
9           NTP                     135
11          NFS                       1
13          BGP                      11
14          SNMP                      4
36          EDONKEY                  95
37          BITTORRENT               10
40          CONTENT_FLASH          8589
48          QQ                        2
51          MAIL_IMAPS                9
60          HTTP_DOWNLOAD           516
64          SSL_NO_CERT             856
67          UNENCRYPED_JABBER        45
68          MSN                   14478
69          OSCAR                     7
70          YAHOO                 21268
81          IP_ICMP                1631
85          IP_OSPF                   5
91          SSL                  404883
92          SSH                     102
114         MSSQL                    21
119         FACEBOOK              29033
120       

In [10]:
# df_main.groupby(['L7Protocol', 'ProtocolName']).size()
# print(df_main.groupby(['L7Protocol', 'ProtocolName']).size().shape)
# df_main['Destination.IP'].value_counts()
df_main = df_main[~df_main['Destination.IP'].astype(str).str.startswith('10.')]
# df_main['Destination.IP'].value_counts()
# df_main.groupby(['L7Protocol', 'ProtocolName']).size()
# df_main.shape

In [11]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(df_main.groupby(['L7Protocol', 'ProtocolName']).size())

L7Protocol  ProtocolName     
5           DNS                    1593
7           HTTP                 605068
11          NFS                       1
13          BGP                       5
36          EDONKEY                  55
37          BITTORRENT                8
40          CONTENT_FLASH          8371
48          QQ                        2
51          MAIL_IMAPS                9
60          HTTP_DOWNLOAD           291
64          SSL_NO_CERT             812
67          UNENCRYPED_JABBER        39
68          MSN                    8395
69          OSCAR                     7
70          YAHOO                 10794
85          IP_OSPF                   3
91          SSL                  359968
92          SSH                      86
114         MSSQL                     2
119         FACEBOOK              15117
120         TWITTER                9495
121         DROPBOX                5126
122         GMAIL                 19151
123         GOOGLE_MAPS             401
124       

In [12]:
value_counts = df_main['ProtocolName'].value_counts()
to_remove = value_counts[value_counts < 100000].index
df_main = df_main[~df_main.ProtocolName.isin(to_remove)]

del value_counts
del to_remove

In [13]:
# grouped = df_main.groupby('ProtocolName')

# def filter_group(group):
#     if len(group) > 100000:
#         return group.sample(n=90000, random_state=1)  # Randomly select 90,000 rows
#     else:
#         return group  # Keep all rows if less than or equal to 100,000

# # Apply the filter function to each group and concatenate the results
# filtered_df = pd.concat([filter_group(group) for _, group in grouped])

# # Reset the index of the resulting DataFrame
# filtered_df.reset_index(drop=True, inplace=True)

# df_main = filtered_df

# del grouped
# del filter_group
# del filtered_df

In [14]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(df_main.groupby(['L7Protocol', 'ProtocolName']).size())

L7Protocol  ProtocolName
7           HTTP            605068
91          SSL             359968
126         GOOGLE          437261
131         HTTP_PROXY      185384
dtype: int64


In [15]:
# grouped = df_main.groupby('ProtocolName')

# def filter_group(group):
#     if len(group) > 180000:
#         return group.sample(n=180000, random_state=1)  # Randomly select 90,000 rows
#     else:
#         return group  # Keep all rows if less than or equal to 100,000

# # Apply the filter function to each group and concatenate the results
# filtered_df = pd.concat([filter_group(group) for _, group in grouped])

# # Reset the index of the resulting DataFrame
# filtered_df.reset_index(drop=True, inplace=True)

# df_main = filtered_df

# del grouped
# del filter_group
# del filtered_df

In [16]:
# df_main['Source.IP'] = df_main['Source.IP'].apply(lambda x: int(x.replace('.', '')))
# df_main['Destination.IP'] = df_main['Destination.IP'].apply(lambda x: int(x.replace('.', '')))

import ipaddress

# Define a function to convert IP addresses to integers
def ip_to_integer(ip):
    try:
        ip_obj = ipaddress.IPv4Address(ip)
        return int(ip_obj)
    except ipaddress.AddressValueError:
        # Handle invalid IP addresses as needed
        return None
    
df_main['Destination.IP'] = df_main['Destination.IP'].apply(ip_to_integer)
df_main['Source.IP'] = df_main['Source.IP'].apply(ip_to_integer)

In [17]:
df_main.head()

,Source.IP,Source.Port,Destination.IP,Destination.Port,Protocol,Flow.Duration,Total.Fwd.Packets,Total.Backward.Packets,Total.Length.of.Fwd.Packets,Total.Length.of.Bwd.Packets,Fwd.Packet.Length.Max,Fwd.Packet.Length.Min,Fwd.Packet.Length.Mean,Fwd.Packet.Length.Std,Bwd.Packet.Length.Max,Bwd.Packet.Length.Min,Bwd.Packet.Length.Mean,Bwd.Packet.Length.Std,Flow.Bytes.s,Flow.Packets.s,Flow.IAT.Mean,Flow.IAT.Std,Flow.IAT.Max,Flow.IAT.Min,Fwd.IAT.Total,Fwd.IAT.Mean,Fwd.IAT.Std,Fwd.IAT.Max,Fwd.IAT.Min,Bwd.IAT.Total,Bwd.IAT.Mean,Bwd.IAT.Std,Bwd.IAT.Max,Bwd.IAT.Min,Fwd.PSH.Flags,Fwd.Header.Length,Bwd.Header.Length,Fwd.Packets.s,Bwd.Packets.s,Min.Packet.Length,Max.Packet.Length,Packet.Length.Mean,Packet.Length.Std,Packet.Length.Variance,FIN.Flag.Count,SYN.Flag.Count,RST.Flag.Count,PSH.Flag.Count,ACK.Flag.Count,URG.Flag.Count,ECE.Flag.Count,Down.Up.Ratio,Average.Packet.Size,Avg.Fwd.Segment.Size,Avg.Bwd.Segment.Size,Fwd.Header.Length.1,Subflow.Fwd.Packets,Subflow.Fwd.Bytes,Subflow.Bwd.Packets,Subflow.Bwd.Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active.Mean,Active.Std,Active.Max,Active.Min,Idle.Mean,Idle.Std,Idle.Max,Idle.Min,L7Protocol,ProtocolName
1,180881159,3128,2886926638,52422,6,1,2,0,12,0.0,6,6,6.000000,0.000000,0,0,0.0,0.000000,1.200000e+07,2.000000e+06,1.000000,0.000000,1.0,1,1.0,1.000000,0.000000,1.0,1.0,0.0,0.000000,0.000000,0.0,0.0,0,40,0,2.000000e+06,0.00000,6,6,6.000000,0.000000,0.000000e+00,0,0,0,0,1,1,0,0,9.000000,6.000000,0.0,40,2,12,0,0,490,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,131,HTTP_PROXY
5,180881158,3128,2886926648,50004,6,105069,136,0,313554,0.0,5840,6,2305.544118,1220.822406,0,0,0.0,0.000000,2.984267e+06,1.294387e+03,778.288889,4540.424367,51842.0,0,105069.0,778.288889,4540.424367,51842.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0,2720,0,1.294387e+03,0.00000,6,5840,2299.372263,1218.469140,1.484667e+06,0,0,0,0,1,0,0,0,2316.279412,2305.544118,0.0,2720,136,313554,0,0,254,-1,135,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,131,HTTP_PROXY
10,180881156,3128,3232281651,57740,6,205118,32,4,6494,3118.0,1460,6,202.937500,464.035311,1460,51,779.5,735.079361,4.686083e+04,1.755087e+02,5860.514286,23425.849096,103196.0,0,205118.0,6616.709677,24836.180077,103196.0,0.0,99727.0,33242.333333,55644.750393,97490.0,400.0,1,640,80,1.560078e+02,19.50097,6,1460,260.837838,514.477864,2.646875e+05,0,1,0,0,1,0,0,0,268.083333,202.937500,779.5,640,32,6494,4,3118,245,255,31,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,131,HTTP_PROXY
11,180881156,3128,3232281651,57740,6,3,5,0,9991,0.0,2920,6,1998.200000,1267.837214,0,0,0.0,0.000000,3.330333e+09,1.666667e+06,0.750000,0.500000,1.0,0,3.0,0.750000,0.500000,1.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0,100,0,1.666667e+06,0.00000,6,2920,1908.500000,1155.078136,1.334206e+06,0,0,0,0,1,0,0,0,2290.200000,1998.200000,0.0,100,5,9991,0,0,353,-1,4,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,131,HTTP_PROXY
12,180881156,3128,3232281651,57740,6,131,3,0,5611,0.0,4145,6,1870.333333,2099.788164,0,0,0.0,0.000000,4.283206e+07,2.290076e+04,65.500000,91.216775,130.0,1,131.0,65.500000,91.216775,130.0,1.0,0.0,0.000000,0.000000,0.0,0.0,0,60,0,2.290076e+04,0.00000,6,4145,1767.750000,1726.702131,2.981500e+06,0,0,0,0,1,0,0,0,2357.000000,1870.333333,0.0,60,3,5611,0,0,353,-1,2,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,131,HTTP_PROXY


In [18]:
x = df_main.drop(columns = ['ProtocolName','Fwd.Packet.Length.Std','Bwd.Packet.Length.Std','Fwd.IAT.Std','Bwd.IAT.Std','Fwd.Header.Length','Bwd.Header.Length','Packet.Length.Std','Packet.Length.Variance','Avg.Fwd.Segment.Size','Avg.Bwd.Segment.Size','Fwd.Header.Length.1','Subflow.Fwd.Packets','Subflow.Fwd.Bytes','Subflow.Bwd.Packets','Subflow.Bwd.Bytes','Init_Win_bytes_forward','Init_Win_bytes_backward','act_data_pkt_fwd','min_seg_size_forward','L7Protocol','Flow.IAT.Std', 'Min.Packet.Length', 'Max.Packet.Length', 'Active.Std', 'Active.Max', 'Active.Min', 'Idle.Std', 'Idle.Max', 'Idle.Min'])
y = df_main['L7Protocol']

In [19]:
# Implement decision tree classifier

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

clf = dtc(max_depth = 20)
clf.fit(x_train, y_train)

# calculate the train and test accuracy

y_train_pred = clf.predict(x_train)
y_test_pred = clf.predict(x_test)

print('Train accuracy: ', accuracy_score(y_train, y_train_pred))
print('Test accuracy: ', accuracy_score(y_test, y_test_pred))

Train accuracy:  0.938245584752595
Test accuracy:  0.9212028834435042


In [20]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           7       0.90      0.97      0.94    120758
          91       0.96      0.91      0.94     72141
         126       0.93      0.96      0.94     87345
         131       0.88      0.69      0.78     37293

    accuracy                           0.92    317537
   macro avg       0.92      0.88      0.90    317537
weighted avg       0.92      0.92      0.92    317537

